# CLOS(Common Lisp Object System)

- Practial Common Lisp > Chapter 16. Object Reorientation: Generic Functions
- Practial Common Lisp > Chapter 17. Object Reorientation: Classes

In [1]:
(ql:quickload :log4cl)
(log:config :debug :NOPRETTY)

To load "log4cl":
  Load 1 ASDF system:
    log4cl

; Loading "log4cl"
.

(:LOG4CL)

# Examples in PCL

## defclass

In [2]:
(defvar *account-number* 0)

(defclass bank-account ()
    ((customer-name
      :initarg :customer-name
      :initform (error "Must supply a customer name."))
     (balance
      :initarg :balance
      :initform 0)
     (account-number
      :initform (incf *account-number*))
     account-type))

;;; initialize-instance
(defmethod initialize-instance :after ((account bank-account) &key opening-bonus-percentage)
  ;; process extra init arguments
  (when opening-bonus-percentage
        (incf (slot-value account 'balance)
              (* (slot-value account 'balance) (/ opening-bonus-percentage 100))))
  ;; for field: account-type
  (let ((balance (slot-value account 'balance)))
    (setf (slot-value account 'account-type)
      (cond
       ((>= balance 100000) :gold)
       ((>= balance 50000) :silver)
       (t :bronze)))))

(defclass checking-account (bank-account)
    (blah))

(defclass saving-account (bank-account)
    (blah))

*ACCOUNT-NUMBER*

#<STANDARD-CLASS COMMON-LISP-USER::BANK-ACCOUNT>

#<STANDARD-METHOD COMMON-LISP:INITIALIZE-INSTANCE :AFTER (BANK-ACCOUNT) {11038C1CF3}>

#<STANDARD-CLASS COMMON-LISP-USER::CHECKING-ACCOUNT>

#<STANDARD-CLASS COMMON-LISP-USER::SAVING-ACCOUNT>

In [3]:
(let ((account (make-instance 'bank-account :customer-name "John Doe" :balance 1000)))
  (print (slot-value account 'customer-name))
  (print (slot-value account 'balance)))

1000


"John Doe" 
1000 

In [4]:
(let ((account (make-instance 'bank-account
                 :customer-name "Sally Sue"
                 :balance 1000
                 :opening-bonus-percentage 5))) ; extra init arguments
  (print (slot-value account 'balance)))

1050


1050 

## Accessor Functions

In [5]:
; reader: balance
(defgeneric balance (account))

(defmethod balance ((account bank-account))
  (slot-value account 'balance))

; setter: customer-name
(defgeneric (setf customer-name) (value account))

(defmethod (setf customer-name) (value (account bank-account))
  (setf (slot-value account 'customer-name) value))

; reader: customer-name
(defgeneric customer-name (account))

(defmethod customer-name ((account bank-account))
  (slot-value account 'customer-name))

#<STANDARD-GENERIC-FUNCTION COMMON-LISP-USER::BALANCE (0)>

#<STANDARD-METHOD COMMON-LISP-USER::BALANCE (BANK-ACCOUNT) {1103AF7BA3}>

#<STANDARD-GENERIC-FUNCTION (COMMON-LISP:SETF COMMON-LISP-USER::CUSTOMER-NAME) (0)>

#<STANDARD-METHOD (COMMON-LISP:SETF COMMON-LISP-USER::CUSTOMER-NAME) (T
                                                                      BANK-ACCOUNT) {1103C22EF3}>

#<STANDARD-GENERIC-FUNCTION COMMON-LISP-USER::CUSTOMER-NAME (0)>

#<STANDARD-METHOD COMMON-LISP-USER::CUSTOMER-NAME (BANK-ACCOUNT) {1103C8D383}>

In [6]:
(let ((account (make-instance 'bank-account :customer-name "John Doe" :balance 1000)))
  (setf (customer-name account) "Sally Sue")
  (print (customer-name account)))

"Sally Sue"


"Sally Sue" 

In [7]:
; use :reader, :writer, :accessor
; also :documentation
(defclass bank-account2 ()
    ((customer-name
      :initarg :customer-name
      :initform (error "Must supply a customer name.")
      :accessor customer-name
      :documentation "Customer's name")
     (balance
      :initarg :balance
      :initform 0
      :reader balance
      :documentation "Current account balance")
     (account-number
      :initform (incf *account-number*)
      :reader account-number
      :documentation "Account number, unique within a bank.")
     (account-type
      :reader account-type
      :documentation "Type of account, one of :gold, :silver or :bronze.")))

(defmethod initialize-instance :after ((account bank-account2) &key)
  ;; for field: account-type
  (let ((balance (slot-value account 'balance)))
    (setf (slot-value account 'account-type)
      (cond
       ((>= balance 100000) :gold)
       ((>= balance 50000) :silver)
       (t :bronze)))))

#<STANDARD-CLASS COMMON-LISP-USER::BANK-ACCOUNT2>

#<STANDARD-METHOD COMMON-LISP:INITIALIZE-INSTANCE :AFTER (BANK-ACCOUNT2) {1103F7ACC3}>

In [8]:
(let ((account (make-instance 'bank-account2 :customer-name "John Doe" :balance 1000)))
  (setf (customer-name account) "Sally Sue")
  (print (customer-name account))
  (print (balance account))
  (print (account-number account))
  (print (account-type account)))

:BRONZE


"Sally Sue" 
1000 
4 
:BRONZE 

In [9]:
; with-slots for :reader
; with-accessor for :accessor

(defparameter *minumum-balance* 5)

(defmethod assess-low-balance-penalty ((account bank-account))
  (with-slots ((bal balance)) account
    (when (< bal *minumum-balance*)
          (decf bal (* bal .01)))))

(defmethod merge-account ((account1 bank-account) (account2 bank-account))
  (with-slots ((balance1 balance)) account1
    (with-slots ((balance2 balance)) account2
      (incf balance1 balance2)
      (setf balance2 0))))

(let ((a1 (make-instance 'bank-account :customer-name "A" :balance 100))
      (a2 (make-instance 'bank-account :customer-name "A" :balance 4)))
  (assess-low-balance-penalty a2)
  (merge-account a1 a2)
  (print (balance a1))
  (print (balance a2)))

*MINUMUM-BALANCE*

#<STANDARD-METHOD COMMON-LISP-USER::ASSESS-LOW-BALANCE-PENALTY (BANK-ACCOUNT) {1104113B43}>

#<STANDARD-METHOD COMMON-LISP-USER::MERGE-ACCOUNT (BANK-ACCOUNT BANK-ACCOUNT) {11041BAE63}>

0


103.96 
0 

## Slots and Inheritance

In [10]:
(defclass foo ()
    ((a :initarg :a :initform "A" :accessor a)
     (b :initarg :b :initform "B" :accessor b)))

(defclass bar (foo)
    ((a :initform (error "Must supply a value for a"))
     (b :initarg :the-b :accessor the-b :allocation :class)))

#<STANDARD-CLASS COMMON-LISP-USER::FOO>

#<STANDARD-CLASS COMMON-LISP-USER::BAR>

## Multiple Inheritance

In [11]:
(defclass money-market-account (checking-account saving-account)
  ())

#<STANDARD-CLASS COMMON-LISP-USER::MONEY-MARKET-ACCOUNT>

## defgeneric, defmethod

In [12]:
(defgeneric withdtaw (account amount)
  (:documentation "Withdraw the specified amount from the account.
  Signal an error if the current balance is less than the amount."))

;; can we named the class defined captialize to distinguish the variable and the class???
(defmethod withdtaw ((account bank-account) amount)
  (log:debug "method withdtaw ((account bank-account) amount) called.")
  (when (< (slot-value account 'balance) amount)
        (error "Account overdrawn."))
  (decf (slot-value account 'balance) amount))

(defun overdraft-account (account)
  (let ((result (make-instance 'bank-account :customer-name "nil")))
    (with-slots ((cn1 customer-name) (b1 balance) (an1 account-number) (at1 account-type)) account
      (with-slots ((cn2 customer-name) (b2 balance) (an2 account-number) (at2 account-type)) result
        (setf cn2 cn1)
        (setf b2 0)
        (setf an2 an1)
        (setf at2 at1)))
    result))

(defmethod withdtaw ((account checking-account) amount)
  (log:debug "method withdtaw ((account checking-account) amount) called.")
  (let ((overdraft (- amount (slot-value account 'balance))))
    (when (plusp overdraft)
          (withdtaw (overdraft-account account) overdraft) ; what does overdraft-account mean???
          (incf (slot-value account 'balance) overdraft)))
  (call-next-method))

(defparameter *account-of-bank-president* (make-instance 'bank-account :customer-name "*account-of-bank-president*"))
(defvar *bank* 1000000)

(defun embezzle (from amount)
  (setf from (- from amount)))

(defmethod withdtaw ((account (eql *account-of-bank-president*)) amount)
  (log:debug "method withdtaw ((account (eql *account-of-bank-president*)) amount) called.")
  (let ((overdraft (- amount (slot-value account 'balance))))
    (when (plusp overdraft)
          (incf (slot-value account 'balance) (embezzle *bank* overdraft))) ; what does embezzle mean???
    ; call what next???
    (call-next-method)))

#<STANDARD-GENERIC-FUNCTION COMMON-LISP-USER::WITHDTAW (0)>

#<STANDARD-METHOD COMMON-LISP-USER::WITHDTAW (BANK-ACCOUNT T) {1104581913}>

OVERDRAFT-ACCOUNT

#<STANDARD-METHOD COMMON-LISP-USER::WITHDTAW (CHECKING-ACCOUNT T) {11047DF7B3}>

*ACCOUNT-OF-BANK-PRESIDENT*

*BANK*

EMBEZZLE

#<STANDARD-METHOD COMMON-LISP-USER::WITHDTAW ((EQL
                                               #<BANK-ACCOUNT {110487A383}>)
                                              T) {1104A1A5F3}>

In [13]:
(let ((account (make-instance 'bank-account :customer-name "John Doe" :balance 10)))
  ; (withdtaw account 100)
  (handler-case (withdtaw account 100)
    (error nil))
  (print (slot-value account 'balance))
  
  (withdtaw *account-of-bank-president* 100)
  (print (slot-value *account-of-bank-president* 'balance)))



<DEBUG> [15:18:52] cl-user 9D84D656CA94403AFFFA47669E77A8FA-298556526.lisp (withdtaw bank-account) - method withdtaw ((account bank-account) amount) called.

10 
<DEBUG> [15:18:52] cl-user 9D84D656CA94403AFFFA47669E77A8FA-298556526.lisp (withdtaw) - method withdtaw ((account (eql *account-of-bank-president*)) amount) called.

<DEBUG> [15:18:52] cl-user 9D84D656CA94403AFFFA47669E77A8FA-298556526.lisp (withdtaw bank-account) - method withdtaw ((account bank-account) amount) called.


999800


999800 

## Method Combinations

In [14]:
(defclass job ()
  ())
(defclass express-job (job)
  ())

(defgeneric priority (job)
  (:documentation "Return the priority at which the job should be run.")
  (:method-combination + :most-specific-last))

(defmethod priority + ((j job)) 
  (log:debug "priority + ((j job))")
  3)
(defmethod priority + ((job express-job)) 
  (log:debug "priority + ((job express-job))")
  10)

(let ((ej (make-instance 'express-job)))
  (priority ej))

#<STANDARD-CLASS COMMON-LISP-USER::JOB>

#<STANDARD-CLASS COMMON-LISP-USER::EXPRESS-JOB>

#<STANDARD-GENERIC-FUNCTION COMMON-LISP-USER::PRIORITY (0)>

#<STANDARD-METHOD COMMON-LISP-USER::PRIORITY + (JOB) {1104EB33C3}>

#<STANDARD-METHOD COMMON-LISP-USER::PRIORITY + (EXPRESS-JOB) {1104F54453}>


<DEBUG> [15:18:52] cl-user 9D84D656CA94403AFFFA47669E77A8FA-666282863.lisp (priority + job) - priority + ((j job))

<DEBUG> [15:18:52] cl-user 9D84D656CA94403AFFFA47669E77A8FA-666282863.lisp (priority + express-job) - priority + ((job express-job))


13

# Examples in ANSI CL

## Object-Oriented Programming

In [15]:
; area with structures and a function
(defstruct rectangle
  height
  width)

(defstruct circle
  radius)

(defun area (x)
  ;; according to the type, behave differently
  (cond ((rectangle-p x)
          (* (rectangle-height x) (rectangle-width x)))
        ((circle-p x)
          (* pi (expt (circle-radius x) 2)))))

(let ((r (make-rectangle)))
  (setf (rectangle-height r) 2
    (rectangle-width r) 3)
  (area r))

RECTANGLE

CIRCLE

AREA

6

In [16]:
; use CLOS
(defclass rectangle-c ()
    (height width))

(defclass circle-c ()
    (radius))

;; behavior in methods

(defmethod area-m ((x rectangle-c))
  (* (slot-value x 'height) (slot-value x 'width)))

(defmethod area-m ((x circle-c))
  (* pi (expt (slot-value x 'radius) 2)))

(let ((r (make-instance 'rectangle-c)))
  (setf (slot-value r 'height) 2
    (slot-value r 'width) 3)
  (area-m r))

#<STANDARD-CLASS COMMON-LISP-USER::RECTANGLE-C>

#<STANDARD-CLASS COMMON-LISP-USER::CIRCLE-C>

#<STANDARD-METHOD COMMON-LISP-USER::AREA-M (RECTANGLE-C) {1101E230A3}>

#<STANDARD-METHOD COMMON-LISP-USER::AREA-M (CIRCLE-C) {1101F09F23}>

6

In [17]:
(defclass colored ()
    (color))

; inheritance
(defclass colored-circle (circle-c colored)
    ())

#<STANDARD-CLASS COMMON-LISP-USER::COLORED>

#<STANDARD-CLASS COMMON-LISP-USER::COLORED-CIRCLE>

## Classes and Intances

In [18]:
(defclass circle2 ()
    (radius center))

; make-instance
(setf c (make-instance 'circle2))
; slot-value
(setf (slot-value c 'radius) 1)

#<STANDARD-CLASS COMMON-LISP-USER::CIRCLE2>

#<CIRCLE2 {11020D5B63}>

1

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::C
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::C


## Slot Properties

In [19]:
; :accessor
; :writer, :reader
(defclass circle2 ()
    ((radius :accessor circle2-radius)
     (center :accessor circle2-center)))

(setf c (make-instance 'circle2))
(setf (circle2-radius c) 1)
(circle2-radius c)

#<STANDARD-CLASS COMMON-LISP-USER::CIRCLE2>

#<CIRCLE2 {11021D8103}>

1

1

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::C
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::C
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::C


In [20]:
; :initform, :initarg
(defclass circle2 ()
    ((radius :accessor circle2-radius
             :initarg :radius
             :initform 1)
     (center :accessor circle2-center
             :initarg :center
             :initform (cons 0 0))))

(setf c (make-instance 'circle2 :radius 3))
(circle2-radius c)
(circle2-center c)

#<STANDARD-CLASS COMMON-LISP-USER::CIRCLE2>

#<CIRCLE2 {11022D18D3}>

3

(0 . 0)

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::C
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::C
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::C


In [21]:
; :allocation :class
; :allocation :instance - default
(defclass tabloid ()
    ((top-story :accessor tabloid-story
                :allocation :class)))

(setf daily-blab (make-instance 'tabloid)
  unsolicited-mail (make-instance 'tabloid))
(setf (tabloid-story daily-blab) 'adultery-of-senator)
(tabloid-story unsolicited-mail)

#<STANDARD-CLASS COMMON-LISP-USER::TABLOID>

#<TABLOID {11023DDFB3}>

ADULTERY-OF-SENATOR

ADULTERY-OF-SENATOR

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::DAILY-BLAB
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::UNSOLICITED-MAIL
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::DAILY-BLAB
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::UNSOLICITED-MAIL


In [22]:
; :documentation
; :type

## Superclasses

In [23]:
(defclass graphic ()
    ((color :accessor graphic-color :initarg :color :initform 'purple)
     (visible :accessor graphic-visible :initarg :visible :initform t)))

(defclass screen-circle (circle2 graphic)
    ())

(graphic-color (make-instance 'screen-circle :color 'red :radius 3))
(graphic-color (make-instance 'screen-circle))


#<STANDARD-CLASS COMMON-LISP-USER::GRAPHIC>

#<STANDARD-CLASS COMMON-LISP-USER::SCREEN-CIRCLE>

RED

PURPLE

## Precedence

In [24]:
(defclass sculpture () (height width depth))
(defclass statue (sculpture) (subject))
(defclass metalwork () (metal-type))
(defclass casting (metalwork) ())
(defclass cast-statue (statue casting) ())

(type-of (make-instance 'cast-statue))
(class-of (make-instance 'cast-statue))
(class-name (class-of (make-instance 'cast-statue)))

(subtypep 'cast-statue 'statue)

#<STANDARD-CLASS COMMON-LISP-USER::SCULPTURE>

#<STANDARD-CLASS COMMON-LISP-USER::STATUE>

#<STANDARD-CLASS COMMON-LISP-USER::METALWORK>

#<STANDARD-CLASS COMMON-LISP-USER::CASTING>

#<STANDARD-CLASS COMMON-LISP-USER::CAST-STATUE>

CAST-STATUE

#<STANDARD-CLASS COMMON-LISP-USER::CAST-STATUE>

CAST-STATUE

T

T

## Generic Functions

In [25]:
(defmethod combine (x y)
  (list x y))

(combine 'a 'b)

(defclass stuff () ((name :accessor name :initarg :name)))
(defclass ice-cream (stuff) ())
(defclass topping (stuff) ())

(defmethod combine ((ic ice-cream) (top topping))
  (format nil "~A ice-cream with ~A topping."
    (name ic)
    (name top)))

(combine (make-instance 'ice-cream :name 'fig)
         (make-instance 'topping :name 'treacle))

#<STANDARD-METHOD COMMON-LISP-USER::COMBINE (T T) {11028CBE53}>

(A B)

#<STANDARD-CLASS COMMON-LISP-USER::STUFF>

#<STANDARD-CLASS COMMON-LISP-USER::ICE-CREAM>

#<STANDARD-CLASS COMMON-LISP-USER::TOPPING>

#<STANDARD-METHOD COMMON-LISP-USER::COMBINE (ICE-CREAM TOPPING) {11029AF913}>

"FIG ice-cream with TREACLE topping."

In [26]:
(combine 23 'skiddoo)

(defmethod combine ((ic ice-cream) x)
  (format nil "~A ice-cream with ~A."
    (name ic)
    x))

(combine (make-instance 'ice-cream :name 'grape)
         (make-instance 'topping :name 'marshmallow))

(combine (make-instance 'ice-cream :name 'clam)
         'reluctance)

(23 SKIDDOO)

#<STANDARD-METHOD COMMON-LISP-USER::COMBINE (ICE-CREAM T) {1102B87D63}>

"GRAPE ice-cream with MARSHMALLOW topping."

"CLAM ice-cream with RELUCTANCE."

In [27]:
(defmethod combine ((x number) (y number))
  (+ x y))

(defmethod combine ((x (eql 'powder)) (y (eql 'spark)))
  'boom)
; redefine
(defmethod combine ((x (eql 'powder)) (y (eql 'spark)))
  'kaboom)

#<STANDARD-METHOD COMMON-LISP-USER::COMBINE (NUMBER NUMBER) {1102D960D3}>

#<STANDARD-METHOD COMMON-LISP-USER::COMBINE ((EQL POWDER) (EQL SPARK)) {1102DF9DC3}>

#<STANDARD-METHOD COMMON-LISP-USER::COMBINE ((EQL POWDER) (EQL SPARK)) {1102EB6483}>

SB-KERNEL:REDEFINITION-WITH-DEFMETHOD: redefining COMBINE (#<SB-MOP:EQL-SPECIALIZER POWDER> #<SB-MOP:EQL-SPECIALIZER SPARK>) in DEFMETHOD


## Auxiliary Methods

In [34]:
(defclass speaker () ())
(defmethod speak ((s speaker) string)
  (log:debug "~A" string))

(speak (make-instance 'speaker) "I'm hungry")

; :before
; :after
(defclass intellectual (speaker) ())
(defmethod speak :before ((i intellectual) string)
  (log:debug "Perhaps "))
(defmethod speak :after ((i intellectual) string)
  (log:debug " in some sense"))
(speak (make-instance 'intellectual) "I'm hungry")

#<STANDARD-CLASS COMMON-LISP-USER::SPEAKER>

#<STANDARD-METHOD COMMON-LISP-USER::SPEAK (SPEAKER T) {1103ADF583}>

SB-KERNEL:REDEFINITION-WITH-DEFMETHOD: redefining SPEAK (#<STANDARD-CLASS COMMON-LISP-USER::SPEAKER>
 #<SB-PCL:SYSTEM-CLASS COMMON-LISP:T>) in DEFMETHOD

<DEBUG> [15:21:25] cl-user 9D84D656CA94403AFFFA47669E77A8FA-1947697567.lisp (speak speaker) - I'm hungry


#<STANDARD-CLASS COMMON-LISP-USER::INTELLECTUAL>

#<STANDARD-METHOD COMMON-LISP-USER::SPEAK :BEFORE (INTELLECTUAL T) {1103C63CA3}>

#<STANDARD-METHOD COMMON-LISP-USER::SPEAK :AFTER (INTELLECTUAL T) {1103CE5C93}>

SB-KERNEL:REDEFINITION-WITH-DEFMETHOD: redefining SPEAK :BEFORE (#<STANDARD-CLASS COMMON-LISP-USER::INTELLECTUAL>
 #<SB-PCL:SYSTEM-CLASS COMMON-LISP:T>) in DEFMETHOD
SB-KERNEL:REDEFINITION-WITH-DEFMETHOD: redefining SPEAK :AFTER (#<STANDARD-CLASS COMMON-LISP-USER::INTELLECTUAL>
 #<SB-PCL:SYSTEM-CLASS COMMON-LISP:T>) in DEFMETHOD

<DEBUG> [15:21:25] cl-user 9D84D656CA94403AFFFA47669E77A8FA-1947697567.lisp (speak :before intellectual) - Perhaps 

<DEBUG> [15:21:25] cl-user 9D84D656CA94403AFFFA47669E77A8FA-1947697567.lisp (speak speaker) - I'm hungry

<DEBUG> [15:21:25] cl-user 9D84D656CA94403AFFFA47669E77A8FA-1947697567.lisp (speak :after intellectual) -  in some sense


In [29]:
; :around
; next-method-p
; call-next-method
(defclass courtier (speaker) ())

(defmethod speak :around ((c courtier) string)
  (format t "Does the King believe that ~A? " string)
  (if (eql (read) 'yes)
      (if (next-method-p) (call-next-method))
      (format t "Indeed, it is a preposterous idea.~%"))
  'bow)

(speak (make-instance 'courtier) "kings will last") ; input: yes

#<STANDARD-CLASS COMMON-LISP-USER::COURTIER>

#<STANDARD-METHOD COMMON-LISP-USER::SPEAK :AROUND (COURTIER T) {110331DA03}>

Does the King believe that kings will last? 

BOW

kings will last

In [30]:
(speak (make-instance 'courtier) "the world is round") ; input: no

Does the King believe that the world is round? 

BOW

Indeed, it is a preposterous idea.


## Method Combination

In [31]:
; standard method combination

; operator method combination
; +, and, append, list, max, min, nconc, or, progn
(defgeneric price (x)
  (:method-combination +))

(defclass jacket () ())
(defclass trousers () ())
(defclass suit (jacket trousers) ())

(defmethod price + ((jk jacket)) 350)
(defmethod price + ((tr trousers)) 200)

(price (make-instance 'suit))

#<STANDARD-GENERIC-FUNCTION COMMON-LISP-USER::PRICE (0)>

#<STANDARD-CLASS COMMON-LISP-USER::JACKET>

#<STANDARD-CLASS COMMON-LISP-USER::TROUSERS>

#<STANDARD-CLASS COMMON-LISP-USER::SUIT>

#<STANDARD-METHOD COMMON-LISP-USER::PRICE + (JACKET) {1103568423}>

#<STANDARD-METHOD COMMON-LISP-USER::PRICE + (TROUSERS) {110356C353}>

550

## Encapsulation

In [32]:
(defpackage "CTR"
  (:use "COMMON-LISP")
  (:export "COUNTER" "INCREMENT" "CLEAR" "GET-STATE"))
(in-package ctr)

(defclass counter () ((state :initform 0)))
(defmethod increment ((c counter))
  (incf (slot-value c 'state)))
(defmethod get-state ((c counter))
  (slot-value c 'state))
(defmethod clear ((c counter))
  (setf (slot-value c 'state) 0))

(let ((c (make-instance 'counter)))
  (slot-value c 'state))

#<PACKAGE "CTR">

#<PACKAGE "CTR">

#<STANDARD-CLASS CTR:COUNTER>

#<STANDARD-METHOD CTR:INCREMENT (COUNTER) {1103810683}>

#<STANDARD-METHOD CTR:GET-STATE (COUNTER) {1103877BC3}>

#<STANDARD-METHOD CTR:CLEAR (COUNTER) {11038DF7D3}>

0

In [ ]:
; *package*
(in-package :cl-user)

(let ((c (make-instance 'ctr:counter)))
  ; (print (slot-value c 'state)) ; missing slot
  (log:debug (ctr:get-state c))
  (ctr:increment c)
  (log:debug (ctr:get-state c))
  (ctr:clear c)
  (log:debug (ctr:get-state c)))

#<PACKAGE "COMMON-LISP-USER">

0


0 
1 
0 